# Group Project: Determining Diamond Cut Grades Using KNN Classification

**Section 009 Group 2**

**Ziqing Wang**<br>**Anna Tao**<br>**Ruby de Lang**

### Introduction

The 4Cs: cut, clarity, color, and carat weight, are internationally accepted standards for assessing the quality of a diamond.  Diamond cut grade is a pivotal factor in determining the beauty and value of a diamond.

The dataset being used reports on the characteristics of diamonds. 
We want to use the KNN classification method to predict the cut grades of diamonds.

The columns of the dataset:
* **carat**: a unit of measurement for a diamond's weight.
* **cut**: cut grades of diamonds, measured in five scales (high to low): Ideal, Premium, Very good, Good, Fair.
* **color**: color is graded on a scale from D (colorless) to Z (light yellow or brown).
* **clarity**: the presence of internal and external flaws within a diamond.
* **depth**: the distance from the table to the culet (the bottom of the diamond).
* **table**: the flat, topmost facet of diamonds.
* **price**: The price of diamonds.
* **x**: the x-dimension of diamonds.
* **y**: the y-dimension of diamonds.
* **z**: the z-dimension of diamonds.

### Method Overview:

Our project wishes to identify whether we can use diamond data to predict the cut of a diamond. To accomplish we will follow the following steps:
![dsci_image.jpeg](attachment:6e619412-1eb7-40ec-8ecc-2fbf3afc9d19.jpeg)  how to insert image???

1. Preliminary Exploratory Data Analysis: Prepare our dataset by reading and wrangling for further analysis.
2. Splitting Data: Split the filtered dataset into a testing and training set. Summarize the distribution of each categorical predictor variables for the training data.
3. Select Predictor Variables: Check for the relationship between each variable and the prediction of cut quality. Eliminate the variables with no correlation.
4. Create a Classification Model: Employ the K-nearest neighbors classification algorithm to identify the optimal K value. After identifying the most suitable value, run the model on the test set to validate predictions.


### 1. Preliminary exploratory data analysis

In [24]:
 install.packages("tidyverse")
 install.packages("cowplot")
 install.packages("tidymodels")
 install.packages("kknn")
install.packages("readr")
library(tidyverse)
library(ggplot2)
library(repr)
library(tidymodels)
library(readr)
library(cowplot)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



**Reading Data from Online Source Into R**

After reading the data from an online source into a CSV file in Jupyter, we have 53940 recorded observations.

In [25]:
diamond_data <- read_csv("https://raw.githubusercontent.com/rubydelang/sonar_data/main/diamonds.csv") 

#head(diamond_data)
diamond_data


Warning message in date_names_lang(date_names):
“restarting interrupted promise evaluation”


ERROR: Error in date_names_lang(date_names): read failed on /opt/conda/lib/R/library/readr/R/sysdata.rdb


**Mutating Data**

For our target variable ( column cut ), we need to mutate the type into a factor. This will allow us to predict the the cut quality.

In [ ]:
mutated_data <- diamond_data |>
mutate(cut = as_factor(cut))
diamond_data <- mutated_data

**Checking for Missing Data** 

The 'na_rows' counts for the number of rows containing missing data, and the result of 0 means we do not have any missing data in this dataset.

In [ ]:
na_rows <- sum(apply(is.na(diamond_data), 1, any))
print(na_rows)
summary(diamond_data)

**Tidy Data**

Now looking at the dataset, each row is a single observation, each column is a single variable, with meaningful column names, and each cell contains only a single value. 
Therefore, the data is already tidy so we do not need take any further actions. <br><br>

### 2. Splitting Data

**Splitting the data into training and testing data**

We have 53940 recorded observations that can be used for analysis. Our next step is the split the data into a training and testing set. We set the proportion to 0.75, this means 75% of our 53940 observations will be used towards the training set, and the remaining observations will be stored for the testing set. We set our seed to 2023 to create replicable randomized results.

In [ ]:
set.seed(2023)
diamond_split <- initial_split(diamond_data, prop = 0.75, strata = cut)
diamond_training <- training(diamond_split)
diamond_testing <- testing(diamond_split)

**Examining the Distribution of Training data**

In the following visualization, we examine the distribution of our target variable 'cut' in the training data set. **THIS SHOULD BE AN EVEN DISTRIBUTION**

In [ ]:
counting_types <- diamond_training |>
    group_by(cut)|>
    summarize(types_count = n()) 

dist <- counting_types |>
    ggplot(aes(x = cut, y = types_count, fill = cut)) +
    geom_bar(stat = "identity") +
    labs(x = "Cut QUality",y = "Patient Count", color = "Cut Quality") +
    ggtitle("Cut Quality Distribution") +
    scale_fill_discrete(name = "Cut Quality", labels = c("1", "2", "4", "5")) +
    theme(text = element_text(size = 20))
dist

**Predictor Distribution**

Below are seven predictor variables for cut quality. This visualization allows us to see the predictor variables that are not evenly distributed. **WHAT ARE THESE PLOTS**

In [32]:
options(repr.plot.width = 13, repr.plot.height = 14)
x_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = x, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "x-dimension of diamonds", color = "cut grades")

y_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = y, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "y-dimension of diamonds", color = "cut grades")

z_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = z, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "z-dimension of diamonds", color = "cut grades")

table_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = table, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "table of diamonds", color = "cut grades")

depth_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = depth, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "depth of diamonds", color = "cut grades")

carat_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = carat, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "carat of diamonds", color = "cut grades")

price_cut_graph <- diamond_training |>
ggplot(aes(x = cut, y = price, color = cut)) +
geom_boxplot() +
labs(x ="cut grades", y = "price of diamonds", color = "cut grades")

plot_grid(x_cut_graph, y_cut_graph, z_cut_graph, table_cut_graph, depth_cut_graph, carat_cut_graph, price_cut_graph, align = "h", ncol = 3)

ERROR: Error in ggplot(diamond_training, aes(x = cut, y = x, color = cut)): object 'diamond_training' not found


The more separate the boxes are, the more accurate prediction it will make. Based on the graph, none of them are strongly associated with the cut grades. We can maximize the prediction accuracy by choosing the relatively associated factors like: depth and table.

### 3. Method Overview:


In reality, the cut grades are classified based on how well the diamond can reflect light. x, y, z, depth, and table are symmetry factors and should be considered as predictors. However, based on the graphs we have plotted, the associations are weak. Therefore, we only choose depth and table as predictors. If the accuracy is not desirable, we will add the x, y, and z in, and compare the accuracy. We will visualize the results by plotting the accuracy graphs. We hypothesize the addition of x, y, and z columns will not affect the accuracy too much because the boxes of each type looks almost identical. 

Our project wishes to identify whether we can use diamond data to predict the cut of a diamond. To accomplish we will follow the following steps:

In reality, the cut grades are classified based on how well the diamond can reflect light. x, y, z, depth, and table are symmetry factors and should be considered as predictors. However, based on the graphs we have plotted, the associations are weak. Therefore, we only choose depth and table as predictors. If the accuracy is not desirable, we will add the x, y, and z in, and compare the accuracy. We will visualize the results by plotting the accuracy graphs. We hypothesize the addition of x, y, and z columns will not affect the accuracy too much because the boxes of each type looks almost identical. 

First, let's choose a reasonable k value to work with. We use tuning to select the best k value, and use cross validation to make sure the accuracy is not an unlucky value. **Warning: This step needs approximately 20 minutes to perform.**

In [31]:
# set.seed(2023)

diamond_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

diamond_recipe <- recipe(cut ~ depth+table, data = diamond_training) |>
step_scale(all_predictors()) |>
step_center(all_predictors())

k_vals <- tibble(neighbors = seq(from = 1, to = 151, by = 10))
diamond_vfold_5 <- vfold_cv(diamond_training, v = 5, strata = cut)

diamond_fit_tune_1 <- workflow() |>
  add_recipe(diamond_recipe) |>
  add_model(diamond_spec) |>
  tune_grid(resamples = diamond_vfold_5, grid = k_vals)

diamond_vfold_tune_metrics_1 <- diamond_fit_tune_1 |>
collect_metrics() |> filter(.metric == "accuracy")

ERROR: Error in rlang::is_missing(data): object 'diamond_training' not found


In [30]:
options(repr.plot.width = 7, repr.plot.height = 6)

accuracy_plot <- ggplot(diamond_vfold_tune_metrics_1, aes(x = neighbors, y = mean)) +
geom_point() +
geom_line() +
labs(x = "# of k", y = "accuracy in %")

accuracy_plot

ERROR: Error in ggplot(diamond_vfold_tune_metrics_1, aes(x = neighbors, y = mean)): object 'diamond_vfold_tune_metrics_1' not found


The graph above suggests that the selection of k between 50-150 would have similar accuracy, so let's pick a relatively smaller number to speed up our remaining calculations. 70 looks to be a good k value to choose.

In [29]:
diamond_spec_final <- nearest_neighbor(weight_func = "rectangular", neighbors = 70) |>
  set_engine("kknn") |>
  set_mode("classification")

diamond_fit_final <- workflow() |>
  add_recipe(diamond_recipe) |>
  add_model(diamond_spec_final) |>
  fit(diamond_training)

diamond_predict <- predict(diamond_fit_final, diamond_testing) |>bind_cols(diamond_testing)

diamond_prediction_accuracy <- diamond_predict |>
  metrics(truth = cut, estimate = .pred_class) |>
  filter(.metric == "accuracy")

diamond_prediction_accuracy

ERROR: Error in is_recipe(recipe): object 'diamond_recipe' not found


In [28]:
set.seed(2023)
diamond_recipe_curiosity <- recipe(cut ~ depth+table+x+y+z, data = diamond_training) |>
step_scale(all_predictors()) |>
step_center(all_predictors())

diamond_fit_tune_curiosity <- workflow() |>
  add_recipe(diamond_recipe_curiosity) |>
  add_model(diamond_spec) |>
  tune_grid(resamples = diamond_vfold_5, grid = k_vals)

diamond_vfold_tune_metrics_curiosity <- diamond_fit_tune_curiosity |>
collect_metrics() |> filter(.metric == "accuracy")

ERROR: Error in rlang::is_missing(data): object 'diamond_training' not found


We can see the final accuracy is 70.8%. Now we want to fulfill our curiosity. Will adding x,y,z as the predictors affect the accuracy, even the relationship graph from section 2 does not show strong relationships. So we can do that by repeating the same steps we used before.

In [27]:
accuracy_curiosity_plot <- ggplot(diamond_vfold_tune_metrics_curiosity, aes(x = neighbors, y = mean)) +
geom_point() +
geom_line() +
labs(x = "# of k", y = "accuracy in %") +
scale_x_continuous(breaks = seq(from = 0, to = 150, by = 10))

accuracy_curiosity_plot

ERROR: Error in ggplot(diamond_vfold_tune_metrics_curiosity, aes(x = neighbors, : object 'diamond_vfold_tune_metrics_curiosity' not found


The graph above suggests the k value to be 10. Now let's test on the testing set.

In [26]:
diamond_spec_curiosity <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) |>
  set_engine("kknn") |>
  set_mode("classification")

diamond_fit_curiosity <- workflow() |>
  add_recipe(diamond_recipe_curiosity) |>
  add_model(diamond_spec_curiosity) |>
  fit(diamond_training)

diamond_predict_curiosity <- predict(diamond_fit_curiosity, diamond_testing) |>bind_cols(diamond_testing)

diamond_prediction_accuracy_curiosity <- diamond_predict_curiosity |>
  metrics(truth = cut, estimate = .pred_class) |>
  filter(.metric == "accuracy")

diamond_prediction_accuracy_curiosity

ERROR: Error in is_recipe(recipe): object 'diamond_recipe_curiosity' not found


In [ ]:
diamond_prediction_accuracy
diamond_prediction_accuracy_curiosity

With only depth and table as our predictors, the accuracy is 70.8%, and after adding the x, y, z as the predictors, the accuracy is increased by 2%. Just like we hypothesized, the addition of the x,y,z does not affect the accuracy too much, even though they are the geometry factors.

### 4 Expected outcomes and significance

The expected outcome is that the cut grade prediction accuracy will be low, as the associations are not strong. In terms of impacts, if the accuracy is desirable, the model can speed up the process of grading the diamonds.

**Future questions** 
1. What factors do affect the cut of the diamond?
2. How can we improve the prediction accuracy? 

### References

1. Why Is A Diamond’s Cut Important?. (n.d.). BRILLIANT EARTH. https://www.brilliantearth.com/en-ca/diamond/buying-guide/cut/